### Salary prediction, episode II: make it actually work (4 points)

Your main task is to use some of the tricks you've learned on the network and analyze if you can improve __validation MAE__. Try __at least 3 options__ from the list below for a passing grade. Write a short report about what you have tried. More ideas = more bonus points. 

__Please be serious:__ " plot learning curves in MAE/epoch, compare models based on optimal performance, test one change at a time. You know the drill :)

You can use either __pytorch__ or __tensorflow__ or any other framework (e.g. pure __keras__). Feel free to adapt the seminar code for your needs. For tensorflow version, consider `seminar_tf2.ipynb` as a starting point.


#### Seminar Legacy Stuff

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("./Train_rev1.zip", compression='zip', index_col=None)
data.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


In [3]:
data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')

text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]
TARGET_COLUMN = "Log1pSalary"

# cast missing values to string "NaN"
data[categorical_columns] = data[categorical_columns].fillna("NaN")
data[text_columns] = data[text_columns].fillna("NaN")

data.sample(3)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,Log1pSalary
62306,68700329,Graduate Sales and Marketing Executive Digita...,Graduate Sales and Marketing Executive Email ...,City London South East,London,NaN,permanent,The Graduate Recruitment Company,Graduate Jobs,17000 - 18000 per annum + Excellent Earnings (...,17500,totaljobs.com,9.770013
128546,70099637,Electrical Design Engineer,Job Title: Electrical Design Engineer Rate: **...,Gloucestershire Gloucestershire England,Gloucestershire,NaN,NaN,Nonstop Engineering,Engineering Jobs,25 -28 per hour,50880,gojobsearch.co.uk,10.837245
165113,71288457,Primary Teachers,Due to the growing requirements for supply sta...,Birmingham,Birmingham,NaN,NaN,Aspire People,Teaching Jobs,105 - 145 per day,30000,MyUkJobs,10.308986


In [4]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()
tokenization = lambda text : " ".join(list(tokenizer.tokenize(text.lower())))

for col in text_columns:
    data[col] = data[col].map(tokenization)

In [5]:
print("Tokenized:")
print(data["FullDescription"][2::100000])
assert data["FullDescription"][2][:50] == 'mathematical modeller / simulation analyst / opera'
assert data["Title"][54321] == 'international digital account manager ( german )'

Tokenized:
2         mathematical modeller / simulation analyst / o...
100002    a successful and high achieving specialist sch...
200002    web designer html , css , javascript , photosh...
Name: FullDescription, dtype: object


In [6]:
from collections import Counter
from itertools import chain
token_counts = Counter()

# Count how many times does each token occur in both "Title" and "FullDescription" in total
for text in chain(data['FullDescription'], data['Title']):
    token_counts.update(text.split())


In [7]:
min_count = 10

# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = sorted(t for t, c in token_counts.items() if c >= min_count)

# Add special tokens for unknown and empty words
UNK, PAD = "UNK", "PAD"
tokens = [UNK, PAD] + tokens

In [8]:
print("Vocabulary size:", len(tokens))
assert type(tokens) == list
assert len(tokens) in range(32000, 35000)
assert 'me' in tokens
assert UNK in tokens
print("Correct!")

Vocabulary size: 34158
Correct!


In [9]:
token_to_id = {token : idx for idx, token in enumerate(tokens)}

In [10]:
assert isinstance(token_to_id, dict)
assert len(token_to_id) == len(tokens)
for tok in tokens:
    assert tokens[token_to_id[tok]] == tok

print("Correct!")

Correct!


In [11]:
UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

def as_matrix(sequences, max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = list(map(str.split, sequences))

    max_len = min(max(map(len, sequences)), max_len or float('inf'))

    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))
    for i,seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix

    return matrix

In [12]:
print("Lines:")
print('\n'.join(data["Title"][::100000].values), end='\n\n')
print("Matrix:")
print(as_matrix(data["Title"][::100000]))

Lines:
engineering systems analyst
hr assistant
senior ec & i engineer

Matrix:
[[10807 30161  2166     1     1]
 [15020  2844     1     1     1]
 [27645 10201    16 15215 10804]]


In [13]:
from sklearn.feature_extraction import DictVectorizer

# we only consider top-1k most frequent companies to minimize memory usage
top_companies, top_counts = zip(*Counter(data['Company']).most_common(1000))
recognized_companies = set(top_companies)
data["Company"] = data["Company"].apply(lambda comp: comp if comp in recognized_companies else "Other")

categorical_vectorizer = DictVectorizer(dtype=np.float32, sparse=False)
categorical_vectorizer.fit(data[categorical_columns].apply(dict, axis=1))

DictVectorizer(dtype=<class 'numpy.float32'>, sparse=False)

In [14]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

Train size =  195814
Validation size =  48954


In [15]:
# GLOBAL CONST
import torch
import torch.nn as nn

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 16
EPOCHS = 5


/home/arqa/Documents/YSDA/NLP/nlp-env/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [16]:
def to_tensors(batch, device):
    batch_tensors = dict()
    for key, arr in batch.items():
        if key in ["FullDescription", "Title"]:
            batch_tensors[key] = torch.tensor(arr, device=device, dtype=torch.int64)
        else:
            batch_tensors[key] = torch.tensor(arr, device=device)
    return batch_tensors


def make_batch(data, max_len=None, word_dropout=0, device=DEVICE):
    """
    Creates a keras-friendly dict from the batch data.
    :param word_dropout: replaces token index with UNK_IX with this probability
    :returns: a dict with {'title' : int64[batch, title_max_len]
    """
    batch = {}
    batch["Title"] = as_matrix(data["Title"].values, max_len)
    batch["FullDescription"] = as_matrix(data["FullDescription"].values, max_len)
    batch['Categorical'] = categorical_vectorizer.transform(data[categorical_columns].apply(dict, axis=1))
    
    if word_dropout != 0:
        batch["FullDescription"] = apply_word_dropout(batch["FullDescription"], 1. - word_dropout)
    
    if TARGET_COLUMN in data.columns:
        batch[TARGET_COLUMN] = data[TARGET_COLUMN].values
    
    return to_tensors(batch, device)

def apply_word_dropout(matrix, keep_prop, replace_with=UNK_IX, pad_ix=PAD_IX,):
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1 - keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])


In [17]:
test_batch = make_batch(data_train[:3], max_len=10)
test_batch

{'Title': tensor([[27645, 29893, 33674,     1,     1,     1,     1],
         [29239,   197, 19175, 20042, 15554, 23162,  4051],
         [10609, 30412, 17746,    33,  8705, 29157,    65]]),
 'FullDescription': tensor([[27645, 29893, 33674, 32939,   982, 27645, 29893, 33674, 16451, 32939],
         [29239,   197, 19175, 20042, 15554, 23162,  4051, 25511,   907,    82],
         [30746, 21956, 20601,  6409, 16451,  8165, 27493,   982, 30412, 17746]]),
 'Categorical': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'Log1pSalary': tensor([ 9.7115, 10.4631, 10.7144])}

In [18]:
def iterate_minibatches(data, batch_size=256, shuffle=True, cycle=False, device=DEVICE, **kwargs):
    """ iterates minibatches of data in random order """
    while True:
        indices = np.arange(len(data))
        if shuffle:
            indices = np.random.permutation(indices)

        for start in range(0, len(indices), batch_size):
            batch = make_batch(data.iloc[indices[start : start + batch_size]], device=device, **kwargs)
            yield batch
        
        if not cycle: break


In [19]:
def test_model(model: nn.Module):
    batch = make_batch(data_train[:100], device=DEVICE)
    criterion = nn.MSELoss()
    
    dummy_pred = model(batch)
    dummy_loss = criterion(dummy_pred, batch[TARGET_COLUMN])
    assert dummy_pred.shape == torch.Size([100])
    assert len(torch.unique(dummy_pred)) > 20, "model returns suspiciously few unique outputs. Check your initialization"
    assert dummy_loss.ndim == 0 and 0. <= dummy_loss <= 250., "make sure you minimize MSE"


In [20]:
def print_metrics(model, data, batch_size=BATCH_SIZE, name="", device=torch.device('cpu'), **kw):
    squared_error = abs_error = num_samples = 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterate_minibatches(data, batch_size=batch_size, shuffle=False, device=device, **kw):
            batch_pred = model(batch)
            squared_error += torch.sum(torch.square(batch_pred - batch[TARGET_COLUMN]))
            abs_error += torch.sum(torch.abs(batch_pred - batch[TARGET_COLUMN]))
            num_samples += len(batch_pred)
    mse = squared_error.detach().cpu().numpy() / num_samples
    mae = abs_error.detach().cpu().numpy() / num_samples
    print("%s results:" % (name or ""))
    print("Mean square error: %.5f" % mse)
    print("Mean absolute error: %.5f" % mae)
    return mse, mae


In [21]:
from tqdm import tqdm

def train_model(model: torch.nn.Module):
    criterion = nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    
    for epoch in range(EPOCHS):
        print(f"epoch: {epoch}")
        model.train()
        for i, batch in tqdm(enumerate(
                iterate_minibatches(data_train, batch_size=BATCH_SIZE, device=DEVICE)),
                total=len(data_train) // BATCH_SIZE
            ):
            pred = model(batch)
            loss = criterion(pred, batch[TARGET_COLUMN])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print_metrics(model, data_val, device=DEVICE)


#### Modules

In [22]:
class SwapAxes(nn.Module):
    def __init__(
        self,
        lax: int,
        rax: int,
    ) -> None:
        super().__init__()
        self._lax = lax
        self._rax = rax

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        return batch.swapaxes(self._lax, self._rax)


In [23]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        return batch.max(dim=2).values


In [24]:
class TextEncoder(nn.Module):
    def __init__(
        self,
        word_embedding: nn.Embedding,
    ) -> None:
        super().__init__()
        self._embed = word_embedding

        self._encoder = nn.Sequential(
            self._embed,
            SwapAxes(1, 2), # swap channels and length
            nn.Conv1d(self.word_emb_dim, self.text_emb_dim, 3),
            GlobalMaxPool1d()
        )

    @property
    def word_emb_dim(self) -> int:
        return self._embed.embedding_dim

    @property
    def text_emb_dim(self) -> int:
        return self.word_emb_dim

    def forward(self, token_batch : torch.Tensor) -> torch.Tensor:
        return self._encoder(token_batch)


In [25]:
class CategoricalEncoder(nn.Module):
    def __init__(
        self,
        n_cat_features: int = len(categorical_vectorizer.vocabulary_),
        out_dim: int = 16
    ) -> None:
        super().__init__()
        self._out_dim = out_dim
        self._layers = nn.Sequential(
            nn.Linear(n_cat_features, out_dim),
            nn.ReLU(),
        )

    @property
    def out_dim(self) -> int:
        return self._out_dim

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        return self._layers(batch)


In [26]:
class SalaryPredictor(nn.Module):
    def __init__(
        self,
        title_encoder       : TextEncoder,
        description_encoder : TextEncoder,
        categorical_encoder : CategoricalEncoder,
    ) -> None:

        super().__init__()
        self._title_encoder       = title_encoder
        self._description_encoder = description_encoder
        self._categorical_encoder = categorical_encoder

        self._regressor = nn.Sequential(
            nn.Linear(self.hid_dim, 1),
            nn.ReLU(),
        )

    @property
    def hid_dim(self) -> int:
        return self._title_encoder.text_emb_dim \
             + self._description_encoder.text_emb_dim \
             + self._categorical_encoder.out_dim

    def forward(self, batch : torch.Tensor) -> torch.Tensor:
        features = torch.cat((
                self._title_encoder      (batch["Title"]),
                self._description_encoder(batch["FullDescription"]),
                self._categorical_encoder(batch["Categorical"])
            ),
            dim = 1
        )
        return self._regressor(features).squeeze()


#### Baseline Evaluation

In [27]:
embedding = nn.Embedding(num_embeddings=len(tokens), embedding_dim=16)

baseline = SalaryPredictor(
    title_encoder       = TextEncoder(word_embedding=embedding),
    description_encoder = TextEncoder(word_embedding=embedding),
    categorical_encoder = CategoricalEncoder(out_dim=16)
).to(DEVICE)

In [28]:
test_model(baseline)

In [29]:
train_model(baseline)

epoch: 0


12239it [02:25, 84.15it/s]                                                                                


 results:
Mean square error: 0.18467
Mean absolute error: 0.33429
epoch: 1


12239it [02:25, 84.39it/s]                                                                                


 results:
Mean square error: 0.15589
Mean absolute error: 0.30475
epoch: 2


...

 results:
Mean square error: 0.14791
Mean absolute error: 0.29861
epoch: 3


12239it [02:27, 82.97it/s]                                                                                


 results:
Mean square error: 0.13843
Mean absolute error: 0.28570
epoch: 4


12239it [02:25, 83.88it/s]                                                                                


 results:
Mean square error: 0.13158
Mean absolute error: 0.27803


### A short report

Please tell us what you did and how did it work.

`<YOUR_TEXT_HERE>`, i guess...

## Recommended options

#### A) CNN architecture

All the tricks you know about dense and convolutional neural networks apply here as well.
* Dropout. Nuff said.
* Batch Norm. This time it's `nn.BatchNorm*`/`L.BatchNormalization`
* Parallel convolution layers. The idea is that you apply several nn.Conv1d to the same embeddings and concatenate output channels.
* More layers, more neurons, ya know...


#### B) Play with pooling

There's more than one way to perform pooling:
* Max over time (independently for each feature)
* Average over time (excluding PAD)
* Softmax-pooling:
$$ out_{i, t} = \sum_t {h_{i,t} \cdot {{e ^ {h_{i, t}}} \over \sum_\tau e ^ {h_{j, \tau}} } }$$

* Attentive pooling
$$ out_{i, t} = \sum_t {h_{i,t} \cdot Attn(h_t)}$$

, where $$ Attn(h_t) = {{e ^ {NN_{attn}(h_t)}} \over \sum_\tau e ^ {NN_{attn}(h_\tau)}}  $$
and $NN_{attn}$ is a dense layer.

The optimal score is usually achieved by concatenating several different poolings, including several attentive pooling with different $NN_{attn}$ (aka multi-headed attention).

The catch is that keras layers do not inlude those toys. You will have to [write your own keras layer](https://keras.io/layers/writing-your-own-keras-layers/). Or use pure tensorflow, it might even be easier :)

#### C) Fun with words

It's not always a good idea to train embeddings from scratch. Here's a few tricks:

* Use a pre-trained embeddings from `gensim.downloader.load`. See last lecture.
* Start with pre-trained embeddings, then fine-tune them with gradient descent. You may or may not download pre-trained embeddings from [here](http://nlp.stanford.edu/data/glove.6B.zip) and follow this [manual](https://keras.io/examples/nlp/pretrained_word_embeddings/) to initialize your Keras embedding layer with downloaded weights.
* Use the same embedding matrix in title and desc vectorizer


#### D) Going recurrent

We've already learned that recurrent networks can do cool stuff in sequence modelling. Turns out, they're not useless for classification as well. With some tricks of course..

* Like convolutional layers, LSTM should be pooled into a fixed-size vector with some of the poolings.
* Since you know all the text in advance, use bidirectional RNN
  * Run one LSTM from left to right
  * Run another in parallel from right to left 
  * Concatenate their output sequences along unit axis (dim=-1)

* It might be good idea to mix convolutions and recurrent layers differently for title and description


#### E) Optimizing seriously

* You don't necessarily need 100 epochs. Use early stopping. If you've never done this before, take a look at [early stopping callback(keras)](https://keras.io/callbacks/#earlystopping) or in [pytorch(lightning)](https://pytorch-lightning.readthedocs.io/en/latest/common/early_stopping.html).
  * In short, train until you notice that validation
  * Maintain the best-on-validation snapshot via `model.save(file_name)`
  * Plotting learning curves is usually a good idea
  
Good luck! And may the force be with you!